In [1]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config

env: JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
env: FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config


In [2]:
%load_ext streaming_jupyter_integrations.magics

Could not find scli config file at '/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/.streaming_config.yml'. Will not load any secrets.
Set env variable JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'


In [3]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081
#%flink_connect 

remote environment has been created.


In [4]:
%%flink_execute_sql
CREATE OR REPLACE TABLE hospital (
  `id` STRING,
  `identifier` ROW (`value` STRING) ARRAY,
  `status` STRING,
  `name` STRING,
  `telecom` ROW (`system` STRING, `value` STRING) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ),
  `position` ROW (
    `longitude` DOUBLE,
    `latitude` DOUBLE
  ),
  `managingOrganization` ROW (
    `identifier` ROW (`value` STRING)
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Location',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [5]:
%%flink_execute_sql
CREATE TABLE hospital_psql (
  `id_organization` STRING,
  `id_location` STRING,
  `status` STRING,
  `name` STRING,
  `telecom` STRING,
  `address_line` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `longitude` DOUBLE,
  `latitude` DOUBLE,
  PRIMARY KEY (`id_organization`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://ep-dark-sea-a20skpa3-pooler.eu-central-1.aws.neon.tech:5432/verceldb',
  'table-name' = 'public.hospital',
  'username' = 'default',
  'password' = 'tUyXvsaMq07d'
);


Job starting...
Job started
Execution successful


In [6]:
%%flink_execute_sql
INSERT INTO hospital_psql
SELECT 
  h.managingOrganization.identifier[1] as id_organization,
  h.id as id_location, 
  status, 
  name, 
  h.telecom[1][1] AS telecom, 
  h.address.line[1] AS address,  
  h.address.city AS city, 
  h.address.state AS state, 
  h.address.country AS country, 
  h.address.postalCode AS postalCode, 
  h.`position`.longitude AS longitude,
  h.`position`.latitude AS latitude
  FROM 
  hospital h WHERE h.managingOrganization.identifier[1] <>'None';

This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
